<b>Definitions</b>

Define the used TANGO devices and constants.
Change list of device proxies and "devices" and "tiles" to actually present and used tiles

Put all devices Online. Tiles must be in Maintenance mode in order to use the test signal generator.

In [ ]:
import tango
import time
import json
import numpy as np

from ska_tango_base.commands import ResultCode
from ska_tango_base.control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    SimulationMode,
    TestMode,
)
# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

# define devices
station = tango.DeviceProxy('low-mccs/station/001')
subrack = tango.DeviceProxy('low-mccs/subrack/0001')
t1 = tango.DeviceProxy('low-mccs/tile/0001')
#t2 = tango.DeviceProxy('low-mccs/tile/0002')
#t3 = tango.DeviceProxy('low-mccs/tile/0003')
#t4 = tango.DeviceProxy('low-mccs/tile/0004')
#t5 = tango.DeviceProxy('low-mccs/tile/0005')
#t6 = tango.DeviceProxy('low-mccs/tile/0006')
#t7 = tango.DeviceProxy('low-mccs/tile/0007')
#t8 = tango.DeviceProxy('low-mccs/tile/0008')

#station.logginglevel=5
devices = [station, subrack, t1,] # t2, t3, t4, t5, t6, t7, t8]
tiles = [t1, ] # t4]

# Put everything online
for d in devices:
    d.adminmode = AdminMode.ONLINE
time.sleep(0.2)
# 
# Tiles must be in Maintenance mode to allow test signal generator
for t in tiles:
    t.adminMode = AdminMode.MAINTENANCE

<b> Local parameters </b>
<ul><li> csp_ingest_ip: IP address of the CSP ingest port</li>
    <li> lmc_ip: IP address of the LMC DAQ system</li>
    <li> input ADC: ADC input channel (0-31, 0-1 for antenna 1, 30-31 for antenna 16)</li>
    <li> csp_rounding: Depending on actual signal level, adjust signal level at channelizer output. Nominal value (4)  is appropriate for a sinewave with RMS input amplitude, as measured by adcPower, in the range 11 to 22. Value is ceil(log2(adcPower/1.4))
    <li> input_frequency: frequency of the input signal. Used to compute the beamformed channel</li></ul>
The beamformer beamforms 8 channels starting at the first even channel equal or lower to this one

In [ ]:
csp_ingest_ip = "10.0.0.98"
lmc_ip = "10.0.0.98"
input_adc = 8
input_frequency = 230e6  # use actual tone frequency
nof_channels = 8
csp_rounding = 4    # adequate for -2:+5 dBm, adcLevel=11:22
start_channel = int(round(input_frequency/800e6*1024))
if start_channel % 2 == 0:
    print(f"Signal is on beamformed channel 0, corresponding to TPM channel {start_channel}")
else: 
    print(f"Signal is on beamformed channel 1, corresponding to TPM channel {start_channel}")
    start_channel = start_channel -1

<b>Turn TPM on</b>
<ul>
    <li>Turn TPM on if not already on.</li>
    <li>Wait for intialization if not already initialised</li> 
    <li>If initialisation succeeds <ul>
        <li>perform initial setup</li>
        <li>start the acquisition.</li></ul>
    <li>Set destination IP addresses</li>
    <li>At the end, print signal level on input ADCs</li></ul>

In [ ]:
t1.on()
t = 0
while not t1.tileprogrammingstate in ['Initialised', 'Synchronised']:
    print(f"{t}: {t1.tileprogrammingstate}")
    time.sleep(2)
    t = t + 2
    if t > 60:
        break
if t > 60:
    print("Initialisation failed")
elif t1.tileprogrammingstate == 'Initialised':
    print(f"{t}: {t1.tileprogrammingstate}")
    t1.ConfigureStationBeamformer(json.dumps({
        "start_channel": 192,
        "n_channels": 8,
        "is_first": True,
        "is_last": True,
    }))
    start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
    t1.StartAcquisition(json.dumps({"start_time": start_time}))
    time.sleep(3)
if t1.tileprogrammingstate == "Synchronised":
    t1.statictimedelays=np.zeros([32],dtype=int)
    t1.channeliserRounding=[csp_rounding]*512
    t1.cspRounding=[0]*384
    t1.SetLmcDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.SetLmcIntegratedDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.Configure40GCore(json.dumps({"core_id": 0, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
    t1.Configure40GCore(json.dumps({"core_id": 1, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
print(f"{t1.fpgaframetime}: Tile is in state {t1.tileprogrammingstate}")
time.sleep(0.5) # to allow for the total power detector to compute the total power
print(f"Input levels: {t1.adcPower}")

Program te test generator to produce null samples except for selected input.
Input ADC can be identified from adcPower attribute above.
Then start the generator.

In [ ]:
t1.StopBeamformer()
print(t1.fpgaframetime)
channels = list(range(32))
channels.remove(input_adc)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+2), RFC_FORMAT)
arguments = json.dumps({
    'tone_frequency': 100.06e6, 
    'tone_amplitude': 0.0, 
    'adc_channels': channels,
    'set_time': start_time})
t1.ConfigureTestGenerator(arguments)
t1.SetBeamformerRegions([start_channel,nof_channels,0,1,0,0,0,0])
cal_gain = 2.0
for antenna in range(16):
    cal_coefs = [antenna*1.0] + ([cal_gain, 0.0, 0.0, 0.0, 0.0, 0.0, cal_gain, 0.0] * 384)
    t1.LoadCalibrationCoefficients(cal_coefs)
t1.ApplyCalibration("")

t1.StartBeamformer(json.dumps({"start_time": start_time}))
time.sleep(2)
print(f"Beamformer running: {t1.isBeamformerRunning}")
current_rounding = t1.channeliserRounding[0]
level = t1.adcPower[input_adc]
chan_level = level * 1.45 * 2**current_rounding
if chan_level > 110 or chan_level < 50:
    rounding = int(np.ceil(np.log2(level/1.4)))
    t1.channeliserRounding = [rounding]*512
    print(f"Adjust channeliser rounding to {rounding}")
else:
    print(f"Keeping old channeliser rounding {current_rounding}")
print(f"Input level: {t1.adcPower[input_adc]}")

In [ ]:
for sample_type in ["raw", "channel", "beam"]:
    print(f"Sending {sample_type} samples to DAQ")
    t1.senddatasamples(json.dumps({"data_type": sample_type}))
    time.sleep(3)